In [1]:
import os

In [2]:
%pwd

'd:\\Data Science\\Customer Churn Prediction with MLOps\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Data Science\\Customer Churn Prediction with MLOps'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: str
    train_data_path: str
    test_data_path: str
    model_name: str
    target_column: str
    
    # XGBoost parameters
    learning_rate: float
    n_estimators: int
    max_depth: int
    gamma: float
    colsample_bytree: float
    subsample: float
    reg_alpha: float
    reg_lambda: float


In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.XGBoost  # Changed from ElasticNet to XGBoost
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            
            # XGBoost parameters
            learning_rate=params.learning_rate,
            n_estimators=params.n_estimators,
            max_depth=params.max_depth,
            gamma=params.gamma,
            colsample_bytree=params.colsample_bytree,
            subsample=params.subsample,
            reg_alpha=params.reg_alpha,
            reg_lambda=params.reg_lambda,
            target_column=schema.name
        )

        return model_trainer_config


In [8]:
import os
import pandas as pd
import joblib
from mlProject import logger
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from dataclasses import dataclass


In [19]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        xgb_model = XGBClassifier(
            learning_rate=self.config.learning_rate,
            n_estimators=self.config.n_estimators,
            max_depth=self.config.max_depth,
            gamma=self.config.gamma,
            colsample_bytree=self.config.colsample_bytree,
            subsample=self.config.subsample,
            reg_alpha=self.config.reg_alpha,
            reg_lambda=self.config.reg_lambda,
            random_state=42
        )
        
        xgb_model.fit(train_x, train_y.values.ravel())
        
        joblib.dump(xgb_model, os.path.join(self.config.root_dir, self.config.model_name))


In [20]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-02-06 15:30:09,282: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-06 15:30:09,284: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-06 15:30:09,286: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-06 15:30:09,287: INFO: common: created directory at: artifacts]
[2025-02-06 15:30:09,289: INFO: common: created directory at: artifacts/model_trainer]
